# Problem Statement

- Have you ever wondered how lenders use various factors such as credit score, annual income, the loan amount approved, tenure, debt-to-income ratio etc. and select your interest rates? 

- The process, defined as ‘risk-based pricing’, uses a sophisticated algorithm that leverages different determining factors of a loan applicant. Selection of significant factors will help develop a prediction algorithm which can estimate loan interest rates based on clients’ information. On one hand, knowing the factors will help consumers and borrowers to increase their credit worthiness and place themselves in a better position to negotiate for getting a lower interest rate. On the other hand, this will help lending companies to get an immediate fixed interest rate estimation based on clients information. Here, your goal is to use a training dataset to predict the loan rate category (1 / 2 / 3) that will be assigned to each loan in our test set.

- You can use any combination of the features in the dataset to make your loan rate category predictions. Some features will be easier to use than others.



![alt text](var_exp.jpg "Title")

# Getting Data and Initial Exploration 

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [64]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [65]:
train.shape

(164309, 14)

In [66]:
test.shape

(109541, 13)

In [67]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164309 entries, 0 to 164308
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Loan_ID                  164309 non-null  int64  
 1   Loan_Amount_Requested    164309 non-null  object 
 2   Length_Employed          156938 non-null  object 
 3   Home_Owner               138960 non-null  object 
 4   Annual_Income            139207 non-null  float64
 5   Income_Verified          164309 non-null  object 
 6   Purpose_Of_Loan          164309 non-null  object 
 7   Debt_To_Income           164309 non-null  float64
 8   Inquiries_Last_6Mo       164309 non-null  int64  
 9   Months_Since_Deliquency  75930 non-null   float64
 10  Number_Open_Accounts     164309 non-null  int64  
 11  Total_Accounts           164309 non-null  int64  
 12  Gender                   164309 non-null  object 
 13  Interest_Rate            164309 non-null  int64  
dtypes: f

In [68]:
train.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


# Exploratory Analysis

In [69]:
train.columns

Index(['Loan_ID', 'Loan_Amount_Requested', 'Length_Employed', 'Home_Owner',
       'Annual_Income', 'Income_Verified', 'Purpose_Of_Loan', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Months_Since_Deliquency', 'Number_Open_Accounts',
       'Total_Accounts', 'Gender', 'Interest_Rate'],
      dtype='object')

In [70]:
def nulls_breakdown(df):
    df_cols = list(df.columns)
    cols_total_count = len(list(df.columns))
    cols_count = 0
    for loc, col in enumerate(df_cols):
        null_count = df[col].isnull().sum()
        total_count = df[col].isnull().count()
        percent_null = round(null_count/total_count*100, 2)
        if null_count > 0:
            cols_count += 1
            print('[iloc = {}] {} has {} null values: {}% null'.format(loc, col, null_count, percent_null))
    cols_percent_null = round(cols_count/cols_total_count*100, 2)
    print('Out of {} total columns, {} contain null values; {}% columns contain null values.'.format(cols_total_count, cols_count, cols_percent_null))

In [71]:
nulls_breakdown(train)

[iloc = 2] Length_Employed has 7371 null values: 4.49% null
[iloc = 3] Home_Owner has 25349 null values: 15.43% null
[iloc = 4] Annual_Income has 25102 null values: 15.28% null
[iloc = 9] Months_Since_Deliquency has 88379 null values: 53.79% null
Out of 14 total columns, 4 contain null values; 28.57% columns contain null values.


## Preprocessing

In [72]:
train['source'] = 'train'
test['source']= 'test'
data = pd.concat([train, test], ignore_index=True)
print(data.shape)

(273850, 15)


In [73]:
data.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate,source
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1.0,train
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3.0,train
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3.0,train
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3.0,train
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1.0,train


In [74]:
data['Loan_Amount_Requested'] = data['Loan_Amount_Requested'].str.replace(',','')

In [75]:
data['Length_Employed'].value_counts()

10+ years    88328
2 years      24420
3 years      21478
< 1 year     21133
5 years      18042
1 year       17336
4 years      16393
7 years      15444
6 years      15351
8 years      13115
9 years      10503
Name: Length_Employed, dtype: int64

In [76]:
data['Length_Employed'].fillna('Unknown', inplace = True)

In [77]:
data['Home_Owner'].value_counts()

Mortgage    117270
Rent         93542
Own          20879
Other           79
None            20
Name: Home_Owner, dtype: int64

In [78]:
data['Home_Owner'].fillna('Unknown', inplace = True)

In [79]:
data['Annual_Income'].fillna(data['Annual_Income'].median(), inplace = True)

In [80]:
data['Months_Since_Deliquency'].fillna(0, inplace = True)

In [81]:
nulls_breakdown(data)

[iloc = 13] Interest_Rate has 109541 null values: 40.0% null
Out of 15 total columns, 1 contain null values; 6.67% columns contain null values.


In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273850 entries, 0 to 273849
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Loan_ID                  273850 non-null  int64  
 1   Loan_Amount_Requested    273850 non-null  object 
 2   Length_Employed          273850 non-null  object 
 3   Home_Owner               273850 non-null  object 
 4   Annual_Income            273850 non-null  float64
 5   Income_Verified          273850 non-null  object 
 6   Purpose_Of_Loan          273850 non-null  object 
 7   Debt_To_Income           273850 non-null  float64
 8   Inquiries_Last_6Mo       273850 non-null  int64  
 9   Months_Since_Deliquency  273850 non-null  float64
 10  Number_Open_Accounts     273850 non-null  int64  
 11  Total_Accounts           273850 non-null  int64  
 12  Gender                   273850 non-null  object 
 13  Interest_Rate            164309 non-null  float64
 14  sour

In [83]:
data['Loan_Amount_Requested'].astype(int)

0          7000
1         30000
2         24725
3         16000
4         17000
          ...  
273845    15000
273846     9600
273847     2000
273848     2125
273849    15000
Name: Loan_Amount_Requested, Length: 273850, dtype: int32

In [84]:
cat_vars = [col for col in data.columns if data[col].dtype == 'O' ]

In [85]:
cat_vars

['Loan_Amount_Requested',
 'Length_Employed',
 'Home_Owner',
 'Income_Verified',
 'Purpose_Of_Loan',
 'Gender',
 'source']

In [86]:
for item in cat_vars:
    print('{} has {} unique values'.format(item, data[item].nunique()))

Loan_Amount_Requested has 1320 unique values
Length_Employed has 12 unique values
Home_Owner has 6 unique values
Income_Verified has 3 unique values
Purpose_Of_Loan has 14 unique values
Gender has 2 unique values
source has 2 unique values


In [87]:
cat_vars.remove('source')

In [88]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat_vars:
    data[col] = le.fit_transform(data[col])

In [89]:
data.shape

(273850, 15)

In [90]:
data.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate,source
0,10000001,1195,10,4,68000.0,2,0,18.37,0,0.0,9,14,0,1.0,train
1,10000002,872,4,0,63000.0,0,2,14.93,0,17.0,12,24,0,3.0,train
2,10000003,647,7,0,75566.4,1,2,15.88,0,0.0,12,16,1,3.0,train
3,10000004,264,10,5,56160.0,1,2,14.34,3,0.0,16,22,1,3.0,train
4,10000005,308,8,3,96000.0,1,2,22.17,1,0.0,19,30,0,1.0,train


In [91]:
cat_vars.remove('Loan_Amount_Requested')

In [92]:
data.shape

(273850, 15)

In [94]:
for item in cat_vars:
    print('{} has {} unique values'.format(item, data[item].nunique()))

Length_Employed has 12 unique values
Home_Owner has 6 unique values
Income_Verified has 3 unique values
Purpose_Of_Loan has 14 unique values
Gender has 2 unique values


In [95]:
data= pd.get_dummies(data, columns=cat_vars)

In [96]:
data.shape

(273850, 47)

In [97]:
data.head()

,Loan_ID,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate,source,...,Purpose_Of_Loan_6,Purpose_Of_Loan_7,Purpose_Of_Loan_8,Purpose_Of_Loan_9,Purpose_Of_Loan_10,Purpose_Of_Loan_11,Purpose_Of_Loan_12,Purpose_Of_Loan_13,Gender_0,Gender_1
0,10000001,1195,68000.0,18.37,0,0.0,9,14,1.0,train,...,0,0,0,0,0,0,0,0,1,0
1,10000002,872,63000.0,14.93,0,17.0,12,24,3.0,train,...,0,0,0,0,0,0,0,0,1,0
2,10000003,647,75566.4,15.88,0,0.0,12,16,3.0,train,...,0,0,0,0,0,0,0,0,0,1
3,10000004,264,56160.0,14.34,3,0.0,16,22,3.0,train,...,0,0,0,0,0,0,0,0,0,1
4,10000005,308,96000.0,22.17,1,0.0,19,30,1.0,train,...,0,0,0,0,0,0,0,0,1,0


In [100]:
#split
pre_train = data.loc[data['source'] == 'train']
pre_test= data.loc[data['source'] == 'test']
#remove source col
pre_train.drop('source',axis =1, inplace= True)
pre_test.drop('source',axis =1, inplace = True)
#creating csvs
pre_train.to_csv("train_modified.csv",index=False)
pre_test.to_csv("test_modified.csv",index=False)

In [ ]:
#model